In [1]:
import pandas as pd
import numpy as np
import math

In [16]:
# raw value: [num, count]
cards = {'AS': [1, 1], 'AH': [1, 1], 'AD': [1, 1], 'AC': [1, 1],
         '2S': [2, 1], '2H': [2, 1], '2D': [2, 1], '2C': [2, 1], 
         '3S': [3, 1], '3H': [3, 1], '3D': [3, 1], '3C': [3, 1], 
         '4S': [4, 1], '4H': [4, 1], '4D': [4, 1], '4C': [4, 1], 
         '5S': [5, 1], '5H': [5, 1], '5D': [5, 1], '5C': [5, 1], 
         '6S': [6, 1], '6H': [6, 1], '6D': [6, 1], '6C': [6, 1], 
         '7S': [7, 1], '7H': [7, 1], '7D': [7, 1], '7C': [7, 1], 
         '8S': [8, 1], '8H': [8, 1], '8D': [8, 1], '8C': [8, 1], 
         '9S': [9, 1], '9H': [9, 1], '9D': [9, 1], '9C': [9, 1], 
         '10S': [10, 1], '10H': [10, 1], '10D': [10, 1], '10C': [10, 1], 
         'JS': [10, 1], 'JH': [10, 1], 'JD': [10, 1], 'JC': [10, 1], 
         'QS': [10, 1], 'QH': [10, 1], 'QD': [10, 1], 'QC': [10, 1], 
         'KS': [10, 1], 'KH': [10, 1], 'KD': [10, 1], 'KC': [10, 1]
        }

In [73]:
game = Game(cards)

In [77]:
# game.game_update('JS')
# game.reset()
# game.cards
game.total

0

In [72]:
# Rules
# dealer hits soft 17
# double after split allowed
# no surrender
# no insurance


# initialize
decks = 1
bankroll = 500

## Attributes
# phase
# n_cards
# dealer
# hand
# htype
# total
# decks_n
# cards
# last_seen
# min_bet
# count

## Pass in
# card
# player_action
# player_sum
# dealer_sum


class Hand:
    """
    Class for hand
    """
    def __init__(self, cards=[], dealer=[], decks=1):
        """
        :param cards: list - list of cards in hand
        """
        self.hand = cards
        self.dealer = dealer
        self.htype = self.hand_type()
        self.total = self.hand_sum()
        self.decks_n = decks
        # self.status = 1

    def hand_type(self):
        """
        determine hand type to apply different rules

        :return: str - "pair" or "soft" or "hard" or "none"
        """
        hand_size = len(self.hand)
        if hand_size <= 1:
            return None

        if hand_size == 2:
            if self.hand[0] == self.hand[1]:
                return "pair"
            else:
                if 1 in self.hand:
                    return "soft"
                else:
                    return "hard"
        elif hand_size > 2:
            if 1 in self.hand:
                if sum(self.hand) > 12:
                    return "hard"
                else:
                    return "soft"
            else:
                return "hard"

    def hand_sum(self):
        """
        calculates max sum of hand

        :return points: int - max sum
        """
        points = sum(self.hand)
        if 1 in self.hand and points + 10 <= 21:
            points += 10
        return points

    def hit(self, card):
        """
        Hit/Double action
        :param card: int - added new card
        :return: None
        """
        self.hand.append(card)
        self.htype = self.hand_type()
        self.total = self.hand_sum()

    def split(self, card):
        """
        Split action
        :param card: int -  added new card
        :return pop: int - split card
        """
        pop = self.hand.pop()
        self.hand.append(card)
        self.htype = self.hand_type()
        self.total = self.hand_sum()
        return pop

    def action(self):
        """
        Recommend action

        :return: str - optimal action
        """
        action_space = {1: "Hit", 2: 'Stand', 3: 'Double', 4: 'Split'}
        basic_df = basic_strat.groupby("deck").get_group(self.decks_n)
        filtered = basic_df[(basic_df['sum'] == self.total) & (basic_df['type'] == self.htype)]
        return action_space[filtered[self.dealer].values[0]]
    

class Round(Hand):
    """
    Round per hand
    """
    def __init__(self):
        self.new_round()
    
    def check_bet(self):
        """
        Betting phase + dealing initial cards
        """
        if self.n_cards == 3:
            self.phase += 1
    
    def check_player(self, player_action, player_sum):
        """
        Player's turn
        player actions: {1: "Hit", 2: 'Stand', 3: 'Double', 4: 'Split'}
        """
        if player_action == 2:
            self.phase += 1
        elif player_sum > 21:
            self.new_round()
    
    def check_dealer(self, dealer_sum):
        if dealer_sum >= 17:
            self.new_round()
            
    def new_round(self):
        self.phase = 0  # 0: betting, 1: player, 2: dealer
        self.n_cards = 0
        super().__init__()
            
    def round_update(self, player_action=1, player_sum=0, dealer_sum=0):
        if self.phase == 0:
            self.check_bet()
        elif self.phase == 1:
            self.check_player(player_action, player_sum)
        elif self.phase == 2:
            self.check_dealer(dealer_sum)


class Game(Round):
    """
    Game per deck
    
    cards: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
    """
    def __init__(self, cards, decks_n=1):
        """
        new game
        """
        super().__init__()
        self.decks_n = decks_n
        self.cards = cards
        self.reset()

    def counter(self, card):
        """
        counting rules

        :param card: str - seen unique card
        :return: none
        """
        num = self.cards[card][0]
        if 2 <= num <= 6:
            self.count -= 1/self.decks_n
        elif num == 10 or num == 1:
            self.count += 1/self.decks_n
        self.last_seen = num

    def game_update(self, card):
        """
        check for unique cards. Moves blackjack logic along with unique cards
        updates available card list
        
        :param card: str - detected card
        """
        if self.cards[card][1] == 1:
            self.cards[card][1] = 0
            self.counter(card)
        
    def bet_size(self):
        """
        Determines how much to bet

        :return: int - betting amount
        """
        if self.count < 2:
            return self.min_bet
        return self.min_bet*(math.floor(self.count)-1)
    
    def reset(self):
        """
        shuffle/new table
        """
        self.count = 0
        self.min_bet = 10
        self.cards = {k:[v[0], 1] for k, v in self.cards.items()}
        self.last_seen = 0


# # debug Class Game
# game1 = Game()
# game1.update(10)
# game1.update(10)

# # get betting size
# print(game1.bet_size())

# # debug Class Hand

# # instantiate hand
# hand1 = Hand([5, 6, 10], dealer=2)
# # get action
# print(hand1.action())
